In [3]:
import numpy as np
import pandas as pd

# Computations of emissions from Business Travels
* __Data source__: Travel expenses from HR-system and travels registered by our travel agent (G-Travel Trd) 

## Train transport
Estimate number of trips with Flytoget and the route Trondheim-Oslo with VY. Use this estimate to calculate the amount of traveled kilometers, and finally the contributed energy usage and amount of emitted CO2-equivalents.

Emission factors: [VY report](https://www.vy.no/globalassets/vy.no/filer-en/arsrapporter/2019-annual-and-sustainability-report.pdf), page 41

In [1]:
train_total_trips = 156
train_total_amount = 35069 

price_vy = 800 # Rough price estimate of the route Trondheim-Oslo, sufficient for this use
price_flytoget = 198

# Distances
dist_flytoget = 48.07
dist_TrdOslo = 485 + 64 # Dovrebanen + Gardermobanen

# Energy and emmissions
kWh_train = 0.0415 # From Vy's yearly report. Energy usage, kWh per seat-kilometer
CO2_train = 12.7/1000 # From Vy's yearly report. 12,7 grammes CO2-equivalents per seat-kilometer.

In [4]:
'''
X = [#flytoget, #TRD-OSLO]
The following lines solves a set of linear equations
'''
A = np.array([[1, 1],[price_flytoget, price_vy]])
B = np.array([train_total_trips, train_total_amount])

X = np.linalg.inv(A).dot(B)

In [5]:
train_km_total = X[0]*dist_flytoget + X[1]*dist_TrdOslo

energy_train = train_km_total * kWh_train   # energy usage in kWh
emission_train = CO2_train * train_km_total # emissions in tonnes CO2-equivalents

In [6]:
verdict_train = pd.DataFrame(columns=["Distance traveled (pkm)" ,"Total CO2 equivalents (tonnes)"])
verdict_train.loc["Train"] = [int(train_km_total), round(emission_train/1000, 2)]
verdict_train

,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Train,10977.0,0.14


## Air transport
Most business travels are booked with G-travel, and they provide data on these travels. Still, a good portion of the travels are 'unregistered', that is simply filed as travel expenses (reiseregning). We compute the total traveled kilometers and the carbon impact by applying medians from the G-travel data on the amount of unregistered travels and summing up.

In [7]:
# Calculate emission from flights 
path = "..."

# Data from G-travel (our travel agency) G-travel calculates the GHG-emission from all our travels.
df_air_travel = pd.read_excel(path + "travels2020.xlsx", sheet_name="Enviorment")

# df_air_travel = df_air_travel[df_air_travel["Airline"] != "Totals"] # Drop summary row
df_air_travel.rename(columns={"Number of One Way Tickets": "Number of tickets"}, inplace=True)

# df_domestic = df_air_travel[df_air_travel["Area"] == "Domestic"].reset_index().drop(columns="index")
# df_nordic = df_air_travel[df_air_travel["Area"] == "Nordic"].reset_index().drop(columns="index")
# df_europe = df_air_travel[df_air_travel["Area"] == "Europe"].reset_index().drop(columns="index")

flights = df_air_travel["Number of tickets"].sum()

# Not all travels are booked via our travel agence. These data is gather from out travel expenses. 
unregistered_fligths = 262

df_air_travel

,From-To City,Number of tickets,Sales Amount Incl. Tax,Average Rate,KM/flight,Kg CO2 /flight,Kg H2O/flight,Gram SO2/flight,Gram NOX/flight,Gram CO/flight,KM Total,Kg CO2,Kg H2O,Gram SO2,Gram NOX,Gram CO
0,Oslo-Trondheim / Trondheim-Oslo,115,124948,1086.504348,362.0,62.77649,25.35662,19.92306,217.3424,315.10,41630.0,7219.29635,2916.01130,2291.15190,24994.3760,36236.50
1,Evenes-Trondheim / Trondheim-Evenes,4,8706,2176.500000,619.0,216.85910,43.37182,34.07786,371.7585,539.04,2476.0,867.43640,173.48728,136.31144,1487.0340,2156.16
2,Stavanger-Trondheim / Trondheim-Stavanger,4,8550,2137.500000,585.0,86.80863,40.99742,32.21226,351.4065,509.53,2340.0,347.23452,163.98968,128.84904,1405.6260,2038.12
3,Nice-Oslo / Oslo-Nice,2,5694,2847.000000,1791.0,249.22300,89.69679,69.96623,717.8479,816.05,3582.0,498.44600,179.39358,139.93246,1435.6958,1632.10
4,Others,2,10281,5140.500000,1791.0,249.22300,89.69679,69.96623,717.8479,816.05,3582.0,498.44600,179.39358,139.93246,1435.6958,1632.10
5,Totals,127,164145,1292.480315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Central data on distance and emissions per flight area, per travel

df_areas = df_air_travel#[df_domestic, df_nordic, df_europe]
central_data = pd.DataFrame()

for df in df_areas:
    t_central_data = pd.DataFrame(index = ["Median", "Average"], columns=df_air_travel.columns)
    t_central_data.drop(columns=["From-To City",  "Number of tickets"], inplace=True)

    for column in t_central_data.columns:
        series = pd.Series(dtype=float)
        for i in range(len(df_areas.index)):
            data = np.array([df_areas[column][i] / df_areas["Number of tickets"][i]]) # for j in range(df_areas["Number of tickets"][i])])
            series = series.append(pd.Series(data))
            
        t_central_data.at["Median", column] = series.median()
        t_central_data.at["Average", column] = series.mean()
#         t_central_data["Area"] = df["Area"][0]
    central_data = central_data.append(t_central_data)

    
central_data


,Sales Amount Incl. Tax,Average Rate,KM/flight,Kg CO2 /flight,Kg H2O/flight,Gram SO2/flight,Gram NOX/flight,Gram CO/flight,KM Total,Kg CO2,Kg H2O,Gram SO2,Gram NOX,Gram CO
Median,2157,539.25,154.75,54.2148,10.843,8.51947,92.9396,134.76,619,216.859,43.3718,34.0779,371.759,539.04
Average,2446.75,848.646,419.03,65.1372,22.2019,17.3424,180.106,216.186,1029.6,172.978,57.8239,45.2291,475.241,599.154
Median,2157,539.25,154.75,54.2148,10.843,8.51947,92.9396,134.76,619,216.859,43.3718,34.0779,371.759,539.04
Average,2446.75,848.646,419.03,65.1372,22.2019,17.3424,180.106,216.186,1029.6,172.978,57.8239,45.2291,475.241,599.154
Median,2157,539.25,154.75,54.2148,10.843,8.51947,92.9396,134.76,619,216.859,43.3718,34.0779,371.759,539.04
Average,2446.75,848.646,419.03,65.1372,22.2019,17.3424,180.106,216.186,1029.6,172.978,57.8239,45.2291,475.241,599.154
Median,2157,539.25,154.75,54.2148,10.843,8.51947,92.9396,134.76,619,216.859,43.3718,34.0779,371.759,539.04
Average,2446.75,848.646,419.03,65.1372,22.2019,17.3424,180.106,216.186,1029.6,172.978,57.8239,45.2291,475.241,599.154
Median,2157,539.25,154.75,54.2148,10.843,8.51947,92.9396,134.76,619,216.859,43.3718,34.0779,371.759,539.04
Average,2446.75,848.646,419.03,65.1372,22.2019,17.3424,180.106,216.186,1029.6,172.978,57.8239,45.2291,475.241,599.154


In [20]:
t_central_data

,Sales Amount Incl. Tax,Average Rate,KM/flight,Kg CO2 /flight,Kg H2O/flight,Gram SO2/flight,Gram NOX/flight,Gram CO/flight,KM Total,Kg CO2,Kg H2O,Gram SO2,Gram NOX,Gram CO
Median,2157,539.25,154.75,54.2148,10.843,8.51947,92.9396,134.76,619,216.859,43.3718,34.0779,371.759,539.04
Average,2446.75,848.646,419.03,65.1372,22.2019,17.3424,180.106,216.186,1029.6,172.978,57.8239,45.2291,475.241,599.154


In [21]:
# central_data_domestic = central_data[central_data.Area == "Domestic"].loc["Median"]
# central_data_nordic = central_data[central_data.Area == "Nordic"].loc["Median"]
# central_data_europe = central_data[central_data.Area == "Europe"].loc["Median"]
central_data_areas = t_central_data.loc['Median']
central_data_areas

Sales Amount Incl. Tax       2157
Average Rate               539.25
KM/flight                  154.75
Kg CO2 /flight            54.2148
Kg H2O/flight              10.843
Gram SO2/flight           8.51947
Gram NOX/flight           92.9396
Gram CO/flight             134.76
KM Total                      619
Kg CO2                    216.859
Kg H2O                    43.3718
Gram SO2                  34.0779
Gram NOX                  371.759
Gram CO                    539.04
Name: Median, dtype: object

In [22]:
# Assume same partition on flight areas in the unregistered flights as in the documented G-travel flights

unregistered_areas_flights = int(unregistered_fligths * df_areas["Number of tickets"].sum() / flights)

# unregistered_domestic_flights = int(unregistered_fligths * df_domestic["Number of tickets"].sum() / flights)
# unregistered_nordic_flights = int(unregistered_fligths * df_nordic["Number of tickets"].sum() / flights)
# unregistered_europe_flights = int(unregistered_fligths * df_europe["Number of tickets"].sum() / flights)

### Converting non-CO2 gases to CO2 equivalents
Using the conversion factors in [this table](https://www.winnipeg.ca/finance/findata/matmgt/documents/2012/682-2012/682-2012_Appendix_H-WSTP_South_End_Plant_Process_Selection_Report/Appendix%207.pdf), we compute the total amount of emitted CO2 equivalents from the different flights.

In [33]:
CO2_eq_map = {"SO2": 0.44/1000, "N2O": 298/1000, "CO": 1.57/1000} # These are all given in grammes. 1/1000 converts them to kg

flight_km_areas = df_areas["KM Total"].sum()  + unregistered_areas_flights * t_central_data["KM Total"]
# flight_km_domestic = df_domestic["KM Total "].sum()  + unregistered_domestic_flights * central_data_domestic["KM Total "]
# flight_km_nordic = df_nordic["KM Total "].sum() + unregistered_nordic_flights * central_data_nordic["KM Total "]
# flight_km_europe = df_europe["KM Total "].sum() + unregistered_europe_flights * central_data_europe["KM Total "]


# Long lines, summing up emissions from G-travel and the unregistered flights, and converting them to CO2-equivalents
flight_CO2eq_areas = df_areas["Kg CO2"].sum() + unregistered_areas_flights*t_central_data["Kg CO2"] + CO2_eq_map["SO2"]*(df_areas["Gram SO2"].sum() + unregistered_areas_flights*t_central_data["Gram SO2"]) + CO2_eq_map["N2O"]*(df_areas["Gram NOX"].sum() + unregistered_areas_flights*t_central_data["Gram NOX"]) + CO2_eq_map["CO"]*(df_areas["Gram CO"].sum() + unregistered_areas_flights*t_central_data["Gram CO"])

# flight_CO2eq_domestic = df_domestic["Kg CO2"].sum() + unregistered_domestic_flights*central_data_domestic["Kg CO2"] + CO2_eq_map["SO2"]*(df_domestic["Gram SO2"].sum() + unregistered_domestic_flights*central_data_domestic["Gram SO2"]) + CO2_eq_map["N2O"]*(df_domestic["Gram NOX"].sum() + unregistered_domestic_flights*central_data_domestic["Gram NOX"]) + CO2_eq_map["CO"]*(df_domestic["Gram CO"].sum() + unregistered_domestic_flights*central_data_domestic["Gram CO"])
# flight_CO2eq_nordic = df_nordic["Kg CO2"].sum() + unregistered_nordic_flights*central_data_nordic["Kg CO2"] + CO2_eq_map["SO2"]*(df_nordic["Gram SO2"].sum() + unregistered_nordic_flights*central_data_nordic["Gram SO2"]) + CO2_eq_map["N2O"]*(df_nordic["Gram NOX"].sum() + unregistered_nordic_flights*central_data_nordic["Gram NOX"]) + CO2_eq_map["CO"]*(df_nordic["Gram CO"].sum() + unregistered_nordic_flights*central_data_nordic["Gram CO"])
# flight_CO2eq_europe = df_europe["Kg CO2"].sum() + unregistered_europe_flights*central_data_europe["Kg CO2"] + CO2_eq_map["SO2"]*(df_europe["Gram SO2"].sum() + unregistered_europe_flights*central_data_europe["Gram SO2"]) + CO2_eq_map["N2O"]*(df_europe["Gram NOX"].sum() + unregistered_europe_flights*central_data_europe["Gram NOX"]) + CO2_eq_map["CO"]*(df_europe["Gram CO"].sum() + unregistered_europe_flights*central_data_europe["Gram CO"])

In [36]:
flight_km_areas

Median     215788
Average    323365
Name: KM Total, dtype: object

In [38]:
verdict_flight = pd.DataFrame(columns=["Distance traveled (pkm)", "Total CO2 equivalents (tonnes)"])


verdict_flight.at["Flight: Areas"] = [int(flight_km_areas[0]), round(flight_CO2eq_areas[0]/1000,1)]

# verdict_flight.at["Flight: Domestic"] = [int(flight_km_domestic), round(flight_CO2eq_domestic/1000,1)]
# verdict_flight.at["Flight: Nordic"] = [int(flight_km_nordic), round(flight_CO2eq_nordic/1000,1)]
# verdict_flight.at["Flight: Europe"] = [int(flight_km_europe), round(flight_CO2eq_europe/1000,1)]

# verdict_flight.at["Air total"] = verdict_flight.sum()

verdict_flight

,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Flight: Areas,215788,104.7


## Car, taxi and bus transport

### Private car transport

In [40]:
# Distribution of cars in Trondheim. Source: SSB table 07849
diesel_cars   = 126661
gasoline_cars = 80311

# Data from internal travel expense report for 2020 
fuel_driven_km = 11443
EV_driven_km = 641

# kg CO2-equivalents per km for private vehicles. Source: https://ofv.no/CO2-utslippet/co2-utslippet-i-januar
emissions_diesel = 0.129
emissions_gasoline = 0.093
# this includes hybrid engines  

# Electric vehicles, power usage, kWh per km. Source: https://www.fjordkraft.no/strom/stromforbruk/elbil/
kWh_EV = 0.2

# Emissions from the Norwegian power grid. CO2-equivalents in kg per kWh. Source: https://www.carbonfootprint.com/docs/2019_06_emissions_factors_sources_for_2019_electricity.pdf
# Outdated source: https://www.nve.no/energibruk-effektivisering-og-teknologier/energibruk/hvor-kommer-strommen-fra/?ref=mainmenu
emissions_kWh = 0.017

diesel_driven_km = diesel_cars/(diesel_cars+gasoline_cars)*fuel_driven_km
gasoline_driven_km = gasoline_cars/(diesel_cars+gasoline_cars)*fuel_driven_km

# kg CO2-equivalents
car_total_km = diesel_driven_km + gasoline_driven_km + EV_driven_km
car_CO2eq = diesel_driven_km*emissions_diesel + gasoline_driven_km*emissions_gasoline + EV_driven_km*kWh_EV*emissions_kWh

car_CO2eq/1000

1.3184789045513403

### Taxi transport

In [42]:
# Price per km, average in Oslo 2019. Sauce: SSB, table no. 11271
price_taxi_2019 = 35
price_taxi = price_taxi_2019*1.02


# Data from travel expense report 2019 (Reiser2019.pdf)
taxi_expence_2020 = 25889

taxi_total_km = int(taxi_expence_2020 / price_taxi)

# CO2-equivalents in kg. Calculation is a weighted average of emissions per km per diesel/gasoline. 
taxi_CO2eq = (diesel_cars/(diesel_cars+gasoline_cars)*emissions_diesel + gasoline_cars/(diesel_cars+gasoline_cars)*emissions_gasoline)*taxi_total_km
taxi_CO2eq

83.3974605260615

### Bus transport
A quick check revealed that the product of all bus travels times the price of the Airport express bus was very close to the total bus expence. We therefore assume that it's sufficient to simplify the calculation by assuming all bus trips are to/from the Airport (Værnes).

In [43]:
# Data from travel expense report 2019 (Reiser2019.pdf)
bus_trips = 40
total_bus_expence = 7438

# Distance to Værnes in kilometers
dist_TRD = 32

bus_total_km = dist_TRD*bus_trips

# Estimating emissions per km in kg using data from Vy bus. Source: 
# https://www.vy.no/globalassets/vy.no/filer-no/arsrapporter/2019-ars--og-barekraftsrapport.pdf
# https://www.vy.no/globalassets/vy.no/filer-no/arsrapporter/2019-arsrapport2.pdf

vy_total_bus_km = 169.4*1000000
vy_total_bus_CO2 = 133513000

# Estimated CO2 emissions per seat-kilometer, assuming 50 seats and 100% utilization
bus_CO2_km = vy_total_bus_CO2 / vy_total_bus_km / (50*1)


bus_CO2eq = bus_CO2_km * bus_total_km
bus_CO2eq

20.17669893742621

### Verdict

In [44]:
verdict_road = pd.DataFrame(columns = ["Distance traveled (pkm)", "Total CO2 equivalents (tonnes)"])
verdict_road.at["Private car"] = [car_total_km, round(car_CO2eq/1000,2)]
verdict_road.at["Taxi"] = [taxi_total_km, round(taxi_CO2eq/1000,2)]
verdict_road.at["Bus"] = [bus_total_km, round(bus_CO2eq/1000,2)]

verdict_road.at["Road total"] = verdict_road.sum()

# Totals

In [45]:
print("Business travel by train: ")
print("verdict_train: ")
display(verdict_train)

Business travel by train: 
verdict_train: 


,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Train,10977.0,0.14


In [46]:
print("Business travel by road:")
print("verdict_road: ")
display(verdict_road)

Business travel by road:
verdict_road: 


,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Private car,12084,1.32
Taxi,725,0.08
Bus,1280,0.02
Road total,14089,1.42


In [47]:
print("Business air travel:")
print("verdict_flight:")
display(verdict_flight)

Business air travel:
verdict_flight:


,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Flight: Areas,215788,104.7


In [48]:
df_total = pd.DataFrame.append(verdict_train, verdict_road)
df_total = df_total.append(verdict_flight)
df_total

,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Train,10977,0.14
Private car,12084,1.32
Taxi,725,0.08
Bus,1280,0.02
Road total,14089,1.42
Flight: Areas,215788,104.7
